In [ ]:
#part 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn import model_selection

In [2]:
df_original = pd.read_csv('C:/Users/Богдан/Downloads/train_40k.csv', encoding='latin-1')

In [3]:
columns = ['Text', 'Cat1']
df = shuffle(df_original[columns])
df = df.iloc[:20000]
df.index = pd.RangeIndex(0, 20000)
#df
df.Cat1 = df.Cat1.astype("category").cat.codes
df.Cat1 =  df.Cat1.astype(np.float64)
df = df[(df.Cat1==0) | (df.Cat1==1)].explode('Text')
df

,Text,Cat1
2,When DD was first born I used Pampers Sensitiv...,0.0
6,"One of my favorites, I spray once on my chest ...",1.0
7,I recieved the tubing with the off-white conne...,0.0
10,One word,1.0
12,"soap, the smell is clean and fresh, and I have...",1.0
...,...,...
19987,At 11 weeks old this is by far the most used i...,0.0
19988,I have owned this product for over 13 years an...,1.0
19990,we bought this for my 2 1/2 year old for our d...,0.0
19996,This is the second one I've purchased. The fir...,1.0


In [4]:
df.Cat1.value_counts()
df_1=df.Text
dff=df
df_2 = df_1.str.replace(r'[^\w\d\s]', ' ')
df_2 = df_2.str.replace(r'\s+', ' ')
df_2 = df_2.str.replace(r'^\s+|\s*?$', ' ')
df_2 = df_2.str.replace(r'\d+(\.\d+)?', 'number')
df_2 = df_2.str.lower()
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df_2 = df_2.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
import nltk
ps = nltk.PorterStemmer()
df_2 = df_2.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))
from nltk.tokenize import word_tokenize
nltk.download('punkt')
all_words = []
for df_1 in df_2:
    words = word_tokenize(df_1)
    for w in words:
        all_words.append(w)
       
all_words = nltk.FreqDist(all_words) 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model=TfidfVectorizer()
tfidf_vec=tfidf_model.fit_transform(df_2)
tfidf_data=pd.DataFrame(tfidf_vec.toarray())
df_5 = df
df = tfidf_data
Cat1 = dff['Cat1']
df['Cat1'] = Cat1


C:\Users\9234~1\AppData\Local\Temp/ipykernel_3416/1511397113.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_1.str.replace(r'[^\w\d\s]', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_3416/1511397113.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'\s+', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_3416/1511397113.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'^\s+|\s*?$', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_3416/1511397113.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'\d+(\.\d+)?', 'number')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Богдан\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] D

In [5]:
X = df.drop('Cat1',axis=1)
Y = Cat1
#sum_list = Y[(Cat1==1) | (Cat1==2)]
Y = tf.keras.utils.to_categorical(Y, 2)
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.2, random_state=42)
print(len(X_train))
print(len(X_test))

4557
1140


In [8]:
model = keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(10199,)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dense(2, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               1305600   
                                                                 
 dense_10 (Dense)            (None, 256)               33024     
                                                                 
 dense_11 (Dense)            (None, 256)               65792     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dense_13 (Dense)            (None, 128)               32896     
                                                                 
 dense_14 (Dense)            (None, 128)              

In [9]:
df_fff = model.fit(X_train, Y_train,epochs = 10, validation_split=0.2, verbose = 2)

Epoch 1/10
114/114 - 1s - loss: 0.3608 - accuracy: 0.8178 - val_loss: 0.2253 - val_accuracy: 0.9550 - 1s/epoch - 13ms/step
Epoch 2/10
114/114 - 1s - loss: 0.0897 - accuracy: 0.9709 - val_loss: 0.1568 - val_accuracy: 0.9572 - 676ms/epoch - 6ms/step
Epoch 3/10
114/114 - 1s - loss: 0.0302 - accuracy: 0.9934 - val_loss: 0.1358 - val_accuracy: 0.9539 - 679ms/epoch - 6ms/step
Epoch 4/10
114/114 - 1s - loss: 0.0117 - accuracy: 0.9970 - val_loss: 0.1699 - val_accuracy: 0.9539 - 687ms/epoch - 6ms/step
Epoch 5/10
114/114 - 1s - loss: 0.0050 - accuracy: 0.9995 - val_loss: 0.1796 - val_accuracy: 0.9518 - 680ms/epoch - 6ms/step
Epoch 6/10
114/114 - 1s - loss: 0.0027 - accuracy: 0.9997 - val_loss: 0.1738 - val_accuracy: 0.9572 - 680ms/epoch - 6ms/step
Epoch 7/10
114/114 - 1s - loss: 0.0019 - accuracy: 0.9997 - val_loss: 0.1663 - val_accuracy: 0.9572 - 682ms/epoch - 6ms/step
Epoch 8/10
114/114 - 1s - loss: 0.0017 - accuracy: 0.9995 - val_loss: 0.1394 - val_accuracy: 0.9496 - 681ms/epoch - 6ms/step
Ep

In [10]:
testt = model.evaluate(X_test, Y_test, verbose=1)
print('Test accuracy:', testt[1])

36/36 [==============================] - 0s 2ms/step - loss: 0.1762 - accuracy: 0.9368
Test accuracy: 0.9368420839309692


In [ ]:
#part 2

In [27]:
import numpy as np 
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
from math import sqrt, ceil
from timeit import default_timer as timer
import tensorflow
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [28]:
with open('C:/Users/Богдан/data2.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

y_test: (12630,)
y_validation: (4410, 43)
x_validation: (4410, 32, 32, 3)
x_train: (86989, 32, 32, 3)
y_train: (86989, 43)
labels: 43
x_test: (12630, 32, 32, 3)


In [29]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 7

h = model.fit(data['x_train'][:2000], data['y_train'][:2000],
              batch_size=5, epochs = epochs,
              validation_data = (data['x_validation'], data['y_validation']),
             verbose=1)

Epoch 1/7
400/400 [==============================] - 33s 84ms/step - loss: 0.5272 - accuracy: 0.8625 - val_loss: 0.9084 - val_accuracy: 0.7481
Epoch 2/7
400/400 [==============================] - 33s 83ms/step - loss: 0.1621 - accuracy: 0.9550 - val_loss: 0.7015 - val_accuracy: 0.8136
Epoch 3/7
400/400 [==============================] - 33s 83ms/step - loss: 0.0712 - accuracy: 0.9815 - val_loss: 0.7608 - val_accuracy: 0.8304
Epoch 4/7
400/400 [==============================] - 34s 84ms/step - loss: 0.0409 - accuracy: 0.9890 - val_loss: 0.9973 - val_accuracy: 0.7912
Epoch 5/7
400/400 [==============================] - 33s 83ms/step - loss: 0.0611 - accuracy: 0.9830 - val_loss: 0.9492 - val_accuracy: 0.7882
Epoch 6/7
400/400 [==============================] - 34s 85ms/step - loss: 0.0466 - accuracy: 0.9860 - val_loss: 0.6799 - val_accuracy: 0.8358
Epoch 7/7
400/400 [==============================] - 33s 84ms/step - loss: 0.0793 - accuracy: 0.9760 - val_loss: 0.9145 - val_accuracy: 0.8161

In [49]:
filters = 5
temp = model.predict(data['x_test'][:200])
temp = np.argmax(temp, axis=1)
temp = np.mean(temp == data['y_test'][:200])  
print('data2 filter {0:d} testing accuracy = {1:.5f}'.format(filters, temp))

data2 filter 5 testing accuracy = 0.86000


In [ ]:
#Part 3

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.utils import shuffle
from sklearn import model_selection
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.embeddings import Embedding

In [154]:
df_original = pd.read_csv('C:/Users/Богдан/Downloads/train_40k.csv', encoding='latin-1')

In [155]:
columns = ['Text', 'Cat1']
df = shuffle(df_original[columns])
df = df.iloc[:5000]
df.index = pd.RangeIndex(0, 5000)
#df
df.Cat1 = df.Cat1.astype("category").cat.codes
df.Cat1 =  df.Cat1.astype(np.float64)
df = df[(df.Cat1==0) | (df.Cat1==1)].explode('Text')

In [156]:
df.Cat1.value_counts()
df_1=df.Text
dff=df
df_2 = df_1.str.replace(r'[^\w\d\s]', ' ')
df_2 = df_2.str.replace(r'\s+', ' ')
df_2 = df_2.str.replace(r'^\s+|\s*?$', ' ')
df_2 = df_2.str.replace(r'\d+(\.\d+)?', 'number')
df_2 = df_2.str.lower()
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df_2 = df_2.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
import nltk
ps = nltk.PorterStemmer()
df_2 = df_2.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))
from nltk.tokenize import word_tokenize
nltk.download('punkt')
all_words = []
for df_1 in df_2:
    words = word_tokenize(df_1)
    for w in words:
        all_words.append(w)
       
all_words = nltk.FreqDist(all_words) 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model=TfidfVectorizer()
tfidf_vec=tfidf_model.fit_transform(df_2)
tfidf_data=pd.DataFrame(tfidf_vec.toarray())
df_5 = df
df = tfidf_data
Cat1 = dff['Cat1']
df['Cat1'] = Cat1
df

C:\Users\9234~1\AppData\Local\Temp/ipykernel_1044/3471169167.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_1.str.replace(r'[^\w\d\s]', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_1044/3471169167.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'\s+', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_1044/3471169167.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'^\s+|\s*?$', ' ')
C:\Users\9234~1\AppData\Local\Temp/ipykernel_1044/3471169167.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2 = df_2.str.replace(r'\d+(\.\d+)?', 'number')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Богдан\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] D

,0,1,2,3,4,5,6,7,8,9,...,5346,5347,5348,5349,5350,5351,5352,5353,5354,Cat1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075914,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
X = df.drop('Cat1',axis=1)
Y = Cat1
Y = tensorflow.keras.utils.to_categorical(Y, 2)
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.2, random_state=42)
steps = 2
print(X_train.shape[0],X_train.shape[1])
print(len(X_test))

1176 5355
294


In [158]:
embedding_vecor_length = 100
model_1 = Sequential()
model_1.add(layers.Embedding(100, embedding_vecor_length, input_length=X_train.shape[1]))
model_1.add(layers.LSTM(120, return_sequences=True))
model_1.add(layers.LSTM(120, return_sequences=False))
model_1.add(Dense(2))
model_1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model_1.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 5355, 100)         10000     
                                                                 
 lstm_25 (LSTM)              (None, 5355, 120)         106080    
                                                                 
 lstm_26 (LSTM)              (None, 120)               115680    
                                                                 
 dense_42 (Dense)            (None, 2)                 242       
                                                                 
Total params: 232,002
Trainable params: 232,002
Non-trainable params: 0
_________________________________________________________________


In [159]:
epochs = 4
batch_size = 32
model_1.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=2)
Y_pred = model_1.predict(X_test)

Epoch 1/4
37/37 - 591s - loss: 0.2789 - accuracy: 0.5017 - 591s/epoch - 16s/step
Epoch 2/4
37/37 - 651s - loss: 0.2540 - accuracy: 0.4626 - 651s/epoch - 18s/step
Epoch 3/4
37/37 - 669s - loss: 0.2522 - accuracy: 0.4813 - 669s/epoch - 18s/step
Epoch 4/4
37/37 - 684s - loss: 0.2506 - accuracy: 0.4949 - 684s/epoch - 18s/step


In [163]:
testt = model_1.evaluate(X_test, Y_test, verbose=1)
print('Test accuracy:', testt[1])

10/10 [==============================] - 34s 3s/step - loss: 0.2501 - accuracy: 0.5102
Test accuracy: 0.5102040767669678
